# NAMD tutorial
- ## Created : 2020/10/03
- ## Created by Yoyo, Jeff @MYTLab
- ### Last update: 2024/8/24 by MYT

MYT*Lab* website: https://mytsai.cc

參考資料：
- [NAMD tutorial ver.2017](https://www.ks.uiuc.edu/Training/Tutorials/namd-index.html)

此教程僅供教學用途

All rights reserved.



### 縮寫:
### VMD TkCon   =  Tk Console

# `(必讀) 解壓縮注意事項 : `
##  >  課程下載  : 檔名為 : `NAMD_Tutorial_2022.zip`  的壓縮檔。
##  >  點開壓縮檔( `NAMD_Tutorial_2022.zip` )將`NAMD_Tutorial_2022 ` 資料夾用滑鼠將其移至`桌面`上。
##  0. 在桌面上點擊檔名為 `NAMD_Tutorial_2022` 的資料夾，並進入。
##  1. 點擊 vmd 的安裝檔( `vmd193win32nocuda`) ，以程式預設的路徑直接安裝 VMD 即可。
##  2.  分子動力學模擬程式( NAMD )的執行檔在資料夾中(`NAMD_Tutorial_2022`)的子資料夾(`NAMD_mac/NAMD_exe`)裡頭，'檔名為 `namd2` 之檔案。 


# A. 生成.psf 檔
## 0. 打開 VMD ( 預設路徑應為: C:\Program Files (x86)\University of Illinois\VMD ; 若是使用Mac則應該是/Users/aa)
## 1. 點VMD Main的【`Extensions`】→【`Tk Console`】→鍵入
##  cd   ~/Desktop/NAMD_Tutorial_2022/Working_dir/
## 2. 點VMD Main的【`file`】→【`New molecule`】→【`Browse...`】→選擇`1UBQ.pdb` 
## →Determine file type 選【`Automaticlally`】→【`Load`】
## 在　`Tk Console`　視窗中"逐行"鍵入(可以用"逐行"複製的方式並貼上)：
    set ubq [atomselect top protein]
    $ubq writepdb ubqp.pdb
##  →產生 ubqp.pdb
## 3. 打開 `JupyterLab` → 在左方視窗中移動路徑到 `Desktop/NAMD_Tutorial_2022/Working_dir/` 中 → 在右方 Launcher 介面中點 Other 區塊中的 `Text File` 的選項，以開啟一個新的 txt 檔案。
## → 在 `JupyterLab` 介面中的`untitled.txt`視窗中鍵入或(複製貼上下列指令) 

    package require psfgen
    topology top_all27_prot_lipid.inp
    pdbalias residue HIS HSE
    pdbalias atom ILE CD1 CD
    segment U {pdb ubqp.pdb}
    coordpdb ubqp.pdb U
    guesscoord
    writepdb ubq.pdb
    writepsf ubq.psf
    
##  →在左方視窗中以滑鼠"右鍵"點擊名為`untitled.txt` 的檔案，選擇 Rename 並改名為 `ubq.pgn`
    
## 4. 在　`Tk Console`　輸入(可以用複製的方式並貼上)：
    source ubq.pgn
    
##  →如此會產生兩個新檔案：ubq.pdb, ubq.psf

## 筆記區:

:

# B. Solvating the Protein
# protein in a water box
## 1. 在　`Tk Console`　"逐行"鍵入(可以用"逐行"複製的方式並貼上)：
    package  require  solvate
    solvate  ubq.psf  ubq.pdb  -t 5 -o ubq_wb
##  →此指令將蛋白質附近（> 5 angstrom）加入一堆水分子形成蛋白質水合模型（正立方水盒子），並產生兩個新檔案：
##   ubq_wb.pdb, ubq_wb.psf
## 2. VMD Main 中已自動新增一 PSF檔案名為 `ubq_wb.psf` 並讀進了一PDB結構檔案名為 `ubq_wb.pdb` 故 Frames 為 1
## 3. 回到VMD　`Tk Console`　鍵入(同樣可以用“逐行”複製的方式並貼上)：
## (注意 'ubq_wb.psf'前面是否有"T"字母)

    set everyone [atomselect top all] 
    measure center $everyone
    
                                >>>>會得一些數值(`center_x` `center_y` `center_z`) 
                                
    measure minmax $everyone

                                >>>>會得一些數值{`min_x` `min_y` `min_z`} {`max_x` `max_y` `max_z`}
                                
##  →上面的指令分別計算整個盒子的 x,y,z “中心” 以及 “大小”

## 筆記區:

:

# C. Ubiquitin in a Water Box: Simulation with Periodic Boundary Conditions　
## 1. 在 `JupyterLab` 在左方視窗中以滑鼠打開名為: `ubq_wb_eq.conf` 的檔案
        檢查以下參數 (最開頭):
      
        structure          ubq_wb.psf      (注意路徑)
        coordinates        ubq_wb.pdb      (注意路徑)

        set outputname     ubq_wb_eq

        >根據剛剛在 B. 3 章節中，得到的{min_x min_y min_z} {max_x max_y max_z}
        
        >計算 max_x - min_x 得 length_x
             max_y - min_y 得 length_y
             max_z - min_z 得 length_z

        >將 {length_x length_y length_z} 以及 {center_x center_y center_z} 的值取小數點至後一位，分別填入以下位置
        (已填好，只需檢查是否與預設的值一樣即可)
        
        
        cellBasisVector1	length_x         0.0	    0.0
        cellBasisVector2         0.0    length_y        0.0
        cellBasisVector3	     0.0         0.0   length_z
             cellOrigin     center_x    center_y   center_z


## 2. 開始使用分子動力學模擬程式 (NAMD)
### 在 `JupyterLab` 在左方視窗中點擊 " `+` " → 在右方 Launcher 介面中點 Other 區塊中的 `Terminal` →
###  鍵入(可複製貼上) :
    (Mac)
    cd ~/Desktop/NAMD_Tutorial_2022/Working_dir/
    
    (Windows)
    cd ~\Desktop\NAMD_Tutorial_2022\Working_dir\ 
###  再鍵入(可複製貼上) :
    (Mac)
    ~/Desktop/NAMD_Tutorial_2022/NAMD_mac/namd2 ./ubq_wb_eq.conf > ubq_wb_eq.log
    
    (Windows)
    ~\Desktop\NAMD_Tutorial_2022\NAMD_exe\namd2 .\ubq_wb_eq.conf > ubq_wb_eq.log

# 備註:   
##  在以上練習中，若想直接看正確結果，可先至資料夾`NAMD_Tutorial`中的子資料夾 `Runed`中找到相對應的輸出檔案。

## 3. 確認檔案是否已輸出
###  可以從桌面上點擊檔名為 `NAMD_Tutorial` 的資料夾，並在裡頭的`Working_dir` 中確認。

## 4. 使用VMD觀察模擬軌跡

### VMD Main 中點  ubq_wb.psf 右鍵 →【`Load Data Into…..`   
###         】→【`Browse...`】→選擇 `ubq_wb_eq.dcd` →按 【`Load`】讀入軌跡
### →　再按一次【`Browse...`】　→　選擇 `ubq_wb.psf`
### →按 【`Load`】讀入 topology 檔案。

### 在 VMD Main 視窗右下角有一個 播放的箭頭，按下去即可動態瀏覽分子軌跡。

## 筆記區:

: